# Learning to use Gym, PyTorch and StableBaselines3 for Reinforcement learning
Very simple notebook for learning how to use the three tools mentioned above for reinforcement learning. I might even throw in Weights & Biases if I'm feeling lucky and then eventually move onto using MuJoCo for better physics simulation

In [8]:
import numpy as np
import sklearn as sk
import matplotlib as plt
from tqdm import tqdm
import gym

MAX_EPISODES = 20
MAX_ITERATIONS = 100

gym.__version__


'0.21.0'

In [9]:
# Start with just the simple cartpole problem

env = gym.make('CartPole-v1')
env.reset()

for idx in range(MAX_EPISODES):
    observation = env.reset()
    for t in range(MAX_ITERATIONS):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        
        if done:
            print(observation)
            print(f"Epsiode finished after {t+1} timesteps")
            break
env.close()

ContextException: Could not create GL context

In [5]:
"""
Information regarding environment:

Observation space:
(4,) array with elements: [position, velocity, angle, angular velocity]

Action space:
(1,) array that is in the range {0,1} (DISCRETE)
"""



'\nInformation regarding environment:\n\nObservation space:\n(4,) array with elements: [position, velocity, angle, angular velocity]\n\nAction space:\n(1,) array that is in the range {0,1} (DISCRETE)\n'